In [13]:
import os

images=[]
lable=[]

lables=['Bishop','King','Knight','Pawn','Queen','Rook']
a=0
for i in os.listdir(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\Chess"):
    c=0
    path=os.path.join(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\Chess",i)
    for j in os.listdir(path):
        if c>=76:
            break
        images.append(os.path.join(path,j))
        lable.append(lables[a])
        c+=1
    a+=1

In [4]:
len(images)

456

In [5]:
len(lable)

456

In [6]:
for i in lables:
    print(lable.count(i))

76
76
76
76
76
76


In [14]:
import pandas as pd
from PIL import Image
import numpy as np

In [21]:
dtf={
    # "X":imagearray,
    "X":images,
    "Y":lable
}
df=pd.DataFrame(dtf)
df=df.sample(frac=1)
df=pd.get_dummies(df,columns=["Y"],dtype=float)
df.columns = df.columns.str.replace("Y_", "")

In [22]:
df.head()

,X,Bishop,King,Knight,Pawn,Queen,Rook
84,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,1.0,0.0,0.0,0.0,0.0
444,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,0.0,0.0,1.0
262,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,1.0,0.0,0.0
452,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,0.0,0.0,1.0
263,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,1.0,0.0,0.0


In [23]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df['X'],df[lables],random_state=69,test_size=0.3)
xtest,xval,ytest,yval=train_test_split(xtest,ytest,random_state=69,test_size=0.5)

In [24]:
import torch
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
import torch.nn as nn

In [25]:
ransform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [26]:
label_row =xtrain.iloc[0]
# label_tensor = torch.tensor(label_row.values.astype(np.float32))
# label_index = torch.argmax(label_tensor).item()
label_row

'C:\\Users\\ASUS\\OneDrive\\Desktop\\code\\datasets\\Chess\\Bishop\\00000062.jpg'

In [27]:
class customdataset(Dataset):
    def __init__(self,x,y,transform=None):
        self.x=x
        self.y=y
        self.transform=transform
    
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, index):
        imgpath=self.x.iloc[index]
        if self.transform:
            image=Image.open(imgpath)
            image=self.transform(image)
        label_row = self.y.iloc[index, 1:7]
        label_tensor = torch.tensor(label_row.values.astype(np.float32))
        label_index = torch.argmax(label_tensor).item()

        return image,label_index
    
      

In [28]:
traindataset=customdataset(x=xtrain,y=ytrain,transform=ransform)
testdataset=customdataset(x=xtest,y=ytest,transform=ransform)
valdataset=customdataset(x=xval,y=yval,transform=ransform)

In [29]:
LR=1e-4
BATCH_SIZE=10
EPOCHES=5

In [30]:
trainloader=DataLoader(traindataset,batch_size=BATCH_SIZE,shuffle=True)
testloader=DataLoader(testdataset,batch_size=BATCH_SIZE,shuffle=True)
valloader=DataLoader(valdataset,batch_size=BATCH_SIZE,shuffle=True)

In [63]:
class Meth(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1=nn.Conv2d(1,32,kernel_size=3,padding=1)
        self.conv2=nn.Conv2d(32,64,kernel_size=3,padding=1)
        self.conv3=nn.Conv2d(64,128,kernel_size=3,padding=1)
        self.pooling=nn.MaxPool2d(2,2)
        self.relu= nn.ReLU()
        self.flatten=nn.Flatten()
        self.linear=nn.Linear((128*16*16),128)
        self.output=nn.Linear(128,len(lables))
    
    def forward(self,x):
        x=self.conv1(x)
        x=self.pooling(x)
        x=self.relu(x)

        x=self.conv2(x) 
        x=self.pooling(x)
        x=self.relu(x)

        x=self.conv3(x) 
        x=self.pooling(x)
        x=self.relu(x)
        
        x=self.flatten(x)
        x=self.linear(x)
        x=self.output(x)

        return x

In [64]:
model=Meth()


In [65]:
import torch.optim as optim
critetion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=LR)

In [66]:

for i in range(EPOCHES):
    ttloss=0
    tacc=0
    ttsample=0

    vtloss=0
    vacc=0
    vtsample=0

    model.train()

    for value,label in trainloader:
        optimizer.zero_grad()
        output=model(value)
        if label.dim() > 1:
            label = torch.argmax(label, dim=1)
        # loss
        batchloss=critetion(output,label)
        ttloss+=batchloss.item()*value.size(0)
        # accuracy
        _, preds = torch.max(output, 1)
        tacc+=(preds == label).sum().item()
        ttsample += label.size(0)

        batchloss.backward()
        optimizer.step()
        # optimizer.zero_grad()

    model.eval()
    with torch.no_grad():
        for value,label in valloader:
            if label.dim() > 1:
                label = torch.argmax(label, dim=1)
            output=model(value)
            # loss
            batchloss=critetion(output,label)
            vtloss+=batchloss.item()*value.size(0)
            # accuracy
           
            _, preds = torch.max(output, 1)
            vacc += (preds == label).sum().item()
            vtsample += label.size(0)

    tatl = ttloss / len(trainloader.dataset)
    ta = (tacc / ttsample)*100 

    vatl = vtloss / len(valloader.dataset)
    va= (vacc / vtsample)*100

    print(f"""Epoch:{i+1}
          
          train_loss:{tatl} ,train_accuracy{ta}
          val_loss: {vatl} ,val_accuracy: {va}""")
    
    print("="*20)

Epoch:1
          
          train_loss:1.6460257989112113 ,train_accuracy24.76489028213166
          val_loss: 1.539860967276753 ,val_accuracy: 37.68115942028986
Epoch:2
          
          train_loss:1.5649598336145047 ,train_accuracy28.526645768025077
          val_loss: 1.510054802549058 ,val_accuracy: 39.130434782608695
Epoch:3
          
          train_loss:1.48674787959335 ,train_accuracy38.24451410658307
          val_loss: 1.5513085576071255 ,val_accuracy: 34.78260869565217
Epoch:4
          
          train_loss:1.402451421026152 ,train_accuracy42.94670846394985
          val_loss: 1.5420603855796482 ,val_accuracy: 31.88405797101449
Epoch:5
          
          train_loss:1.2962684481868922 ,train_accuracy48.90282131661442
          val_loss: 1.5343757435895395 ,val_accuracy: 34.78260869565217
